In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os

In [3]:
os.chdir('/content/gdrive/MyDrive')

In [ ]:
! git clone https://github.com/Shraeyas/Plagiarism-Detection.git

fatal: destination path 'Plagiarism-Detection' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/gdrive/MyDrive/Plagiarism-Detection/dataset_20news')

In [ ]:
! wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

--2022-05-17 07:33:13--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 173.48.209.137
Connecting to qwone.com (qwone.com)|173.48.209.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz’

20news-19997.tar.gz 100%[===================>]  16.53M  10.8MB/s    in 1.5s    

2022-05-17 07:33:15 (10.8 MB/s) - ‘20news-19997.tar.gz’ saved [17332201/17332201]



In [ ]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!tar -xvf 20news-19997.tar.gz

In [ ]:
!pwd

/content/gdrive/MyDrive/Plagiarism-Detection/dataset_20news


In [4]:
import os
import numpy as np
import pandas as pd

In [5]:
os.chdir('/content/gdrive/MyDrive/Plagiarism-Detection')

In [6]:
files = []
dataset_directory = os.path.join (os.getcwd (), "dataset_20news")
def get_files () :
    for (root, dirs, file) in os.walk (dataset_directory) :
        for f in file :
            if ".ipynb_checkpoints" not in root :
                files.append (os.path.join (root, f))

In [7]:
get_files()
print (files[2000:2003])
print ("Total no. of files : ", len (files))

['/content/gdrive/MyDrive/Plagiarism-Detection/dataset_20news/20_newsgroups/soc.religion.christian/20606', '/content/gdrive/MyDrive/Plagiarism-Detection/dataset_20news/20_newsgroups/soc.religion.christian/20579', '/content/gdrive/MyDrive/Plagiarism-Detection/dataset_20news/20_newsgroups/soc.religion.christian/20581']
Total no. of files :  19997


In [8]:
files_text = []
for file in files :
    files_text.append (open (os.path.join ("dataset", file), 'r', encoding = "cp437").read ())
print (len (files_text))

19997


In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer

nltk.download ('punkt')
nltk.download ('stopwords')
lancaster = LancasterStemmer()

def pre_process (_sentences) :
    en_stops = set (stopwords.words ('english')) # Remove Stopwords
    processed_sentences = []

    for sentence in _sentences :
        sentence = re.sub (r'[^\w\s]','', sentence) # Remove special characters
        lower_sen = sentence.lower () # Convert the sentence to Lowercase
        token_sen = word_tokenize (lower_sen) # Tokenize the sentence

        new_sentence = []
        for word in token_sen:
            if word not in en_stops and word.isnumeric() == False : # Remove words that contain only digits
                word = lancaster.stem (word) # Perform Stemming
                new_sentence.append (word)

        if len (new_sentence) > 3 :
            processed_sentences.append (new_sentence)
    return processed_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
documents = []
for file_text in files_text :
    sentences = file_text.split (".")
    documents.append (pre_process (sentences))

In [12]:
document_df = []
for i, document in enumerate (documents) :
    for sentence in document :
        sen = " ".join (sentence)
        document_df.append ([i, sen])

#Part 2

In [13]:
document_df = pd.DataFrame (document_df, columns = ["document_id", "sentences"])

document_df.head ()

,document_id,sentences
0,0,hardw subject want slot card vga hddcontr
1,0,bitnet dat apr gmt org forschungszentr juelich...
2,0,help pleas mail ich344djukfa11 ich344zam001
3,0,de thank lot martin muel
4,1,ketil albertsentih subject dcc minidisc next d...


In [ ]:
document_df.to_csv (os.path.join ("dataset_processed", "documents_processed.csv"), index = False)

In [ ]:
import os
import numpy as np
import pandas as pd

documents = pd.read_csv (os.path.join ("dataset_processed", "documents_processed.csv"))
documents.head ()

,document_id,sentences
0,0,hardw subject want slot card vga hddcontr
1,0,bitnet dat apr gmt org forschungszentr juelich...
2,0,help pleas mail ich344djukfa11 ich344zam001
3,0,de thank lot martin muel
4,1,ketil albertsentih subject dcc minidisc next d...


In [ ]:
from gensim.models.doc2vec import TaggedDocument
tagged_documents = []
for index, row in documents.iterrows ():
    tagged_document = TaggedDocument (words = row['sentences'].split (" "), tags = [row['document_id']])
    tagged_documents.append (tagged_document)

tagged_documents[: 1]

[TaggedDocument(words=['hardw', 'subject', 'want', 'slot', 'card', 'vga', 'hddcontr'], tags=[0])]

In [ ]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec (vector_size = 100, window = 3, min_count = 5, workers = 8, epochs = 10, alpha = 0.03, min_alpha = 0.002)
model.build_vocab (tagged_documents)
model.train (tagged_documents, total_examples = model.corpus_count, epochs = model.epochs)


model_path = os.path.join ("models", "doc2vec_20news.bin")
model.save (model_path)

#Partie 3

In [9]:
test_data_files = pd.read_csv (os.path.join ("dataset_test", "file_information.csv"))
test_data_files.head()

,File,Task,Category
0,g0pA_taska.txt,a,non
1,g0pA_taskb.txt,b,cut
2,g0pA_taskc.txt,c,light
3,g0pA_taskd.txt,d,heavy
4,g0pA_taske.txt,e,non


In [14]:
files = test_data_files['File']
files.head ()

0    g0pA_taska.txt
1    g0pA_taskb.txt
2    g0pA_taskc.txt
3    g0pA_taskd.txt
4    g0pA_taske.txt
Name: File, dtype: object

In [16]:
files_text = []
for file in files :
    files_text.append (open (os.path.join ("dataset_test", file), 'r', encoding = "cp437").read ())

print (len (files_text))

100


In [17]:
files_text[0]

'Inheritance is a basic concept of Object-Oriented Programming where\nthe basic idea is to create new classes that add extra detail to\nexisting classes. This is done by allowing the new classes to reuse\nthe methods and variables of the existing classes and new methods and\nclasses are added to specialise the new class. Inheritance models the\nΓÇ£is-kind-ofΓÇ¥ relationship between entities (or objects), for example,\npostgraduates and undergraduates are both kinds of student. This kind\nof relationship can be visualised as a tree structure, where ΓÇÿstudentΓÇÖ\nwould be the more general root node and both ΓÇÿpostgraduateΓÇÖ and\nΓÇÿundergraduateΓÇÖ would be more specialised extensions of the ΓÇÿstudentΓÇÖ\nnode (or the child nodes). In this relationship ΓÇÿstudentΓÇÖ would be\nknown as the superclass or parent class whereas, ΓÇÿpostgraduateΓÇÖ would\nbe known as the subclass or child class because the ΓÇÿpostgraduateΓÇÖ\nclass extends the ΓÇÿstudentΓÇÖ class.\n\nInheritance can occur 

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer

nltk.download ('punkt')
nltk.download ('stopwords')
lancaster = LancasterStemmer()

def pre_process (_sentences) :
    en_stops = set (stopwords.words ('english')) # Remove Stopwords
    processed_sentences = []

    for sentence in _sentences :
        sentence = re.sub (r'[^\w\s]','', sentence) # Remove special characters
        lower_sen = sentence.lower () # Convert the sentence to Lowercase
        token_sen = word_tokenize (lower_sen) # Tokenize the sentence

        new_sentence = []
        for word in token_sen:
            if word not in en_stops and word.isnumeric() == False : # Remove words that contain only digits
                word = lancaster.stem (word) # Perform Stemming
                new_sentence.append (word)

        if len (new_sentence) > 3 :
            processed_sentences.append (new_sentence)
    return processed_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
documents = []
for i, file_text in enumerate (files_text) :
    sentences = file_text.split (".")
    documents.append (pre_process (sentences))

flat_documents = []
for document in documents :
    flat_documents.append ([item for sublist in document for item in sublist])

In [ ]:
flat_documents = []
for document in documents :
    flat_documents.append ([item for sublist in document for item in sublist])

flat_documents[50]


In [ ]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load (os.path.join ("models", "doc2vec_20news.bin"))

In [21]:
vectors = []
for flat_document in flat_documents :
    v = model.infer_vector (flat_document)
    vectors.append (v)

vectors[0]

array([ 0.459561  ,  0.44098863,  0.01395981, -0.08142444, -0.1877713 ,
       -0.6884915 ,  0.87731653,  0.3138414 , -0.7625324 ,  0.7391486 ,
        0.5972845 , -0.15721916, -0.4485428 , -0.3587655 , -0.65047365,
       -0.70440286,  0.8617288 ,  1.0535734 ,  0.55897045,  1.031368  ,
       -0.21487868,  1.4900851 ,  0.02619372,  0.1252698 , -0.07208861,
       -0.6429147 ,  0.26081046, -0.85352933,  0.16524227,  0.5438938 ,
       -0.422844  ,  0.08931097, -0.61227214,  0.2983841 ,  0.01111475,
       -0.75005764, -0.28919452,  1.1132137 , -0.6087024 , -1.3923271 ,
        0.25730127,  0.6935172 , -0.27907005, -0.17331374, -0.7452827 ,
       -0.2815595 , -0.7140893 , -0.8969468 , -0.28018418,  0.38572538,
       -0.34368753, -0.47100228, -0.8309469 ,  1.2174155 ,  0.9387454 ,
        0.00490246,  0.09779351,  0.806482  ,  0.29262668, -0.96498   ,
        0.11666247,  0.42060006, -0.21556845, -0.12516075,  0.07221487,
       -0.09698968, -1.0019437 ,  1.1136503 , -0.3054498 , -0.26

In [22]:
vectors = []
for flat_document in flat_documents :
    v = model.infer_vector (flat_document)
    vectors.append (v)

In [ ]:
vectors[0]

In [23]:
vectors[50]

array([ 5.4017675e-01,  1.3841707e-03,  9.9747455e-01,  1.0378610e-01,
       -5.2764899e-01, -7.5554079e-01,  6.8725520e-01, -5.3918594e-01,
       -1.0983536e+00,  1.0396587e+00, -1.7384616e-01, -2.5308359e-01,
       -1.0586176e+00, -5.8416758e-02, -8.2025915e-01, -9.9681479e-01,
        7.4471658e-01,  1.1881481e+00,  2.9778105e-01, -2.9860941e-01,
       -8.1080355e-02,  1.1581085e+00,  5.4067302e-02,  4.1962206e-01,
       -1.1878569e-01,  2.7506074e-01, -3.5900265e-01, -7.1910495e-01,
        2.6839826e-01, -5.9287101e-01,  7.2111554e-02,  2.5380561e-01,
       -4.8200423e-01, -2.1141039e-01,  3.5971111e-01, -1.6847949e+00,
       -3.2752961e-01,  1.0430273e+00,  5.1831953e-02, -7.1380514e-01,
       -1.3497765e-01, -2.1131252e-01, -2.1760505e-01, -7.6748627e-01,
       -1.1054372e+00,  6.7040510e-02, -1.4436541e-02, -4.4102037e-01,
       -5.3707194e-01,  7.5000741e-02, -4.6353191e-01,  1.8822452e-02,
       -3.6683562e-01,  1.7447660e+00,  1.9671468e-01, -4.1105020e-01,
      

In [24]:
from scipy import spatial
values = []
for i in range (len (vectors)) :
    for j in range (i + 1, len (vectors)) :
        result = 1 - spatial.distance.cosine (vectors[i], vectors[j])
        values.append ([result, i + 1, j + 1])

values.sort()

##Sentence wise similarity

We'll calculate sentence wise similarity documents
The following function takes 2 documents as input and returns cosine_distance and the start and end index in both the documents for the particular pair

In [26]:
def get_similarity (doc_0, doc_1) :
    sentences_dict = {}
    
    sentences_0 = []
    sentences_0 = doc_0.split (".")
    sentences_0_processed = []
    sentences_0_processed.append (pre_process (sentences_0))
    vectors_0 = []
    for i, sentence in enumerate (sentences_0_processed[0]) :
        v = model.infer_vector (sentence)
        for match in re.finditer (sentences_0[i], doc_0) :
            vectors_0.append ([v, match.start (), match.end ()]) 
    
    sentences_1 = []
    sentences_1 = doc_1.split (".")
    sentences_1_processed = []
    sentences_1_processed.append (pre_process (sentences_1))
    vectors_1 = []
    for i, sentence in enumerate (sentences_1_processed[0]) :
        v = model.infer_vector (sentence)
        for match in re.finditer (sentences_1[i], doc_1) :
            vectors_1.append ([v, match.start (), match.end ()])
    
    values = []
    for i in range (len (vectors_0)) :
        for j in range (len (vectors_1)) :
            result = 1 - spatial.distance.cosine (vectors_0[i][0], vectors_1[j][0])
            values.append ([result, [vectors_0[i][1], vectors_0[i][2]], [vectors_1[j][1], vectors_1[j][2]]])
    
    return values

In [27]:
vals = get_similarity (files_text[43], files_text[50])



In [28]:
vals.sort (reverse = True)



In [29]:
vals[0 : 5]

[[0.6331886649131775, [713, 788], [845, 962]],
 [0.6071925163269043, [713, 788], [963, 1049]],
 [0.5274089574813843, [713, 788], [1169, 1307]],
 [0.4100756049156189, [789, 876], [845, 962]],
 [0.40640178322792053, [1650, 1651], [1169, 1307]]]